In [ ]:
from imgphon import helmet 
from imgphon import imgphon as iph
import os, subprocess, glob, re
import numpy as np
import audiolabel
import cv2
import dlib
from imutils.face_utils import FaceAligner
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = [x*2 for x in plt.rcParams['figure.figsize']] # double plot window size

### Development area

In [ ]:
# tilt normalization on lips directly?

In [ ]:
# tSNE
from sklearn.manifold import TSNE

n_tsne_comp = 2
X_embedded = TSNE(n_components=n_tsne_comp).fit_transform(analysis)

X_df = pd.DataFrame(X_embedded)
X_df.columns = ["D{}".format(d+1) for d in range(n_tsne_comp)]
X_df = X_df.assign(phone=pd.Series(md['uniquePhone']).values)
df = df.assign(phase=pd.Series(md['phase']).values)
X_df.head(5)
sns.lmplot(x="D1", y="D2", data=X_df, 
             hue="phone", fit_reg=False, size=5)

### Direct manipulations of video files (legacy)

In [ ]:
# extract and display a single landmark-annotated video frame
video = 'SN12_AA032001.MXF'
time = '00:00:10.900'

frame = get_video_frame(video,time)
shape = detect_landmarks(frame)
color_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(draw_landmarks(color_corrected,shape))

In [ ]:
# extract a frame corresponding to the desired timepoint of an audio file (here, from a TextGrid) 
# and extracts a polygon of the lip shape
babb = 'vowels/P1.MXF'
babb_tg = os.path.splitext(babb)[0] + ".TextGrid"
pm = audiolabel.LabelManager(from_file=babb_tg, from_type="praat")
v = pm.tier('phone').search(vre)[0] # should only be one match per file; TODO check while running
midpoint= v.center
ffmpeg_time = str(round(midpoint,3))
frame = get_video_frame(babb,ffmpeg_time)
shape = detect_landmarks(frame)
color_corrected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
plt.imshow(draw_landmarks(color_corrected,shape))

image = lip_mask(frame,shape)
plt.imshow(crop_center(image))

In [ ]:
# clip a short stretch of video from the longer videos typical of ultrasound acquisitions
exp_movie = '12-video/SN12_AA032001.MXF'
exp_snippet = '12-video/SN12_AA032001_short.MXF'
fname = os.path.split(exp_movie)[1]
basename = os.path.splitext(fname)[0]

# get a snippet of the MXF
snippet_args = ['ffmpeg', '-ss', '00:00:30', 
                '-i', exp_movie, 
                "-t", "00:00:08", 
                "-vcodec", "copy", 
                "-acodec", "copy", 
                exp_snippet]
subprocess.check_call(snippet_args)

In [ ]:
exp_movie = 'vowels/POO1.MXF'
fname = os.path.split(exp_movie)[1]
basename = os.path.splitext(fname)[0]

subprocess.check_call(['ffmpeg','-loglevel','8','-i',exp_snippet,'-vf','fps=20','img_%05d_f.bmp'])
movie = cv2.VideoCapture('img_%05d_f.bmp')

frame_num = 1

while(movie.isOpened()):
    ret, frame = movie.read()
    if (ret==False):   # the file is finished
        break

    # detect face region and draw landmarks on the image.
    shape = detect_landmarks(frame)
    out_image = draw_landmarks(frame,shape)
    
    # write the image to a .bmp file, with zero-padding to ensure the frames are input 
    cv2.imwrite('{0:05d}g.bmp'.format(frame_num), out_image)
    frame_num += 1
        
# cleanup
cv2.destroyAllWindows()

# make a gif (a bit slowed down by default) from the output bmps
subprocess.check_call(['convert', '*g.bmp', basename+'.gif'])

# remove the input bmps
for bmp in glob.glob("*g.bmp"):
    os.remove(bmp)
for bmp in glob.glob("*f.bmp"):
    os.remove(bmp)

Working up an anonymizing function

In [ ]:
det = dlib.get_frontal_face_detector()
pred = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')
algn = FaceAligner(pred, desiredFaceWidth = 256)

In [ ]:
def landmark_plot_prep(target_dir, frame_idx):
    bmp = os.listdir(target_dir)[frame_idx] # get -70th file in dir and ...
    frame = cv2.imread(os.path.join(target_dir, bmp)) # read in as ndarray
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # change color map to RGB for display
    marks = iph.detect_landmarks(frame, detector=det, predictor=pred) # detect landmarks using imgphon
    return rgb, marks

In [ ]:
#import importlib
#importlib.reload(imgphon)

rgb,marks = landmark_plot_prep('../40-lip-frames',-50) # get -50th frame for Subject 40 and...

# plot raw data on left; data annotated with detected facial landmarks on right
plt.subplot(1,2,1)
plt.imshow(rgb)
plt.subplot(1,2,2)
plt.imshow(iph.draw_landmarks(rgb,marks,anonymize=True,line_width=3))
plt.tight_layout()
plt.show()

In [ ]:
# get bottom of box using nose
nose_start,nose_end = face_utils.FACIAL_LANDMARKS_IDXS['nose']
max_y = max([marks[i][1] for i in range(nose_start, nose_end)]) # get lowest point in nose

# get top of box using eyebrows
eyebrows_start = face_utils.FACIAL_LANDMARKS_IDXS['right_eyebrow'][0]
eyebrows_end = face_utils.FACIAL_LANDMARKS_IDXS['left_eyebrow'][1]
min_y = min([marks[i][1] for i in range(eyebrows_start, eyebrows_end)])

# get sides of box using jaw
jaw_start,jaw_end = face_utils.FACIAL_LANDMARKS_IDXS['jaw']
max_x = max([marks[i][0] for i in range(jaw_start, jaw_end)])
min_x = min([marks[i][0] for i in range(jaw_start, jaw_end)])

# pad out the blurred area
# TODO change to integer pixel values (will throw error in notebooks)
width = max_x - min_x
height = max_y - min_y
min_y -= 0.4*height
min_x -= 0.1*width
max_y += 0.1*height
max_x += 0.1*width

# replace the area around the selected landmarks with a blurred version of the area
upper_face = rgb[min_y:max_y, min_x:max_x]
blur = cv2.GaussianBlur(sub_face,(101, 101), 30)
anon_image = rgb.copy()
anon_image[min_y:min_y+upper_face.shape[0], min_x:min_x+upper_face.shape[1]] = blur
plt.imshow(iph.draw_landmarks(anon_image,marks))

#### 

In [ ]:
# old get_frame function, if ever needed
def get_frame(filename, frame_idx, frame_dim_1, frame_dim_2):
    """
      Inputs: filename, index of frame, frame width and height
      Outputs: ndarray containing frame
    """
    data_fmt = 'I' * np.int(frame_dim_1 * frame_dim_2) # unsigned int (I) times dimensions of output image
    framesize = frame_dim_1 * frame_dim_2 * 4 # multiply by 4 to get number of bytes (4 bytes/pixel)
    frame_start = frame_idx * framesize
    with open(filename, 'rb') as fh:
        fh.seek(frame_start) # move pointer to start of frame
        packed_data = fh.read(framesize) # take a frame-sized chunk at this coord
        try:
            unpacked_data = struct.unpack(data_fmt, packed_data) # conversion to array
        except:
            print("No data at position {}".format(frame_idx)) # catch out-of-range indices
            rdata = None
        else:
            data = np.array(unpacked_data)
            rdata = data[np.arange(frame_dim_1 * frame_dim_2)].reshape(frame_dim_1, frame_dim_2).T
            rdata = rdata.astype(np.uint8) # for plotting
            
        return(rdata)